# 2. Profiling - Compare data after preprocessing

The first step in a data preparation pipeline is the exploratory data analysis (EDA). In a nutshell, data exploration and data cleansing are hand-to-hand and both are mutually iterative steps.

*But what does data exploration includes? And how to make a better data exploration giving we are building a customer churn model classifier?*

Data exploration includes both univariate and bivariate analysis and ranges from univariate statistics and frequency distributions to correlations, cross-tabulation, and characteristic analysis. add here detail about pandas-profiling and data exploration in general (re-use the sentence above)


In [1]:
!pip install pandas-profiling==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 8.0 MB/s eta 0:00:0000:01
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 3.4.0
    Uninstalling pandas-profiling-3.4.0:
      Successfully uninstalled pandas-profiling-3.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 1.12.0 requires pandas-profiling==3.4.0, but you have pandas-profiling 3.5.0 which is incompatible.


### Import needed packages

In [2]:
import os
import json
import warnings
import pandas as pd
from pathlib import Path

from ydata.labs.datasources import DataSources

from pandas_profiling import ProfileReport

from functions.saving_functions import read_file

INFO: 2022-12-03 00:23:15,768 Pandas backend loaded 1.2.3
INFO: 2022-12-03 00:23:15,775 Numpy backend loaded 1.21.2
INFO: 2022-12-03 00:23:15,776 Pyspark backend NOT loaded
INFO: 2022-12-03 00:23:15,777 Python backend loaded
INFO: 2022-12-03 00:23:16,477 generated new fontManager


In [3]:
pipeline_parameters = read_file(file_path='pipeline_parameters.pkl')

og_source = os.environ.get('OG_Datasource_id', "a6433f80-b070-4028-8008-0d02dabf3d94")
proc_source = pipeline_parameters['preprocessed_id']

title_og = os.environ.get('ORIGINAL_TITLE', "Original dataset")
title_comp = os.environ.get('COMPARE_TITLE', "Processed dataset")

## Read the sources of data

In [4]:
#get the dataset from the reading step
og_dataset = DataSources.get(uid=og_source).read().to_pandas()
proc_dataset = DataSources.get(uid=proc_source).read().to_pandas()

## Profiling the data

In [5]:
profile_title = title_og
compare_title = title_comp

og_profile = ProfileReport(df=og_dataset, title=profile_title)
#og_profile.config.html.navbar_show = False

compare_profile = ProfileReport(df=proc_dataset, title=compare_title)
#compare_profile.config.html.navbar_show = False

compare = og_profile.compare(compare_profile)

/opt/conda/lib/python3.8/site-packages/pandas_profiling/compare_reports.py:177: UserWarning: The datasets being profiled have a different set of columns. Only the left side profile will be calculated.
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

## Create pipeline outputs

In [6]:
import json

profile_pipeline_output = {
    'outputs' : [
        {
      'type': 'web-app',
      'storage': 'inline',
      'source': compare.to_html(),
    }
    ]
  }

with open('mlpipeline-ui-metadata.json', 'w') as metadata_file:
    json.dump(profile_pipeline_output, metadata_file)

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]